In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import collections
import matplotlib.pyplot as plt
import codecs
import time
import os

In [ ]:
url = 'http://mattmahoney.net/dc/'


def maybe_download(filename, expected_bytes):
  #Download a file if not present, and make sure it's the right size.
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

def read_data(filename):
    if not os.path.exists('enwik8'):
        f = zipfile.ZipFile(filename)
        for name in f.namelist():
            full_text = tf.compat.as_str(f.read(name))
        f.close()
        """f = open('enwik8', 'w')
        f.write(text.encode('utf8'))
        f.close()"""
    else:
        f = open('enwik8', 'r')
        full_text = f.read().decode('utf8')
        f.close()
    return full_text
        
    f = codecs.open('enwik8', encoding='utf-8')
    text = f.read()
    f.close()
    return text

def check_not_one_byte(text):
    not_one_byte_counter = 0
    max_character_order_index = 0
    min_character_order_index = 2**16 
    present_characters = [0]*256
    number_of_characters = 0
    for i in range(len(text)):
        if ord(text[i]) > 255:
            not_one_byte_counter += 1 
        if len(present_characters) <  ord(text[i]):
            present_characters.extend([0]*(ord(text[i]) - len(present_characters) + 1))
            present_characters[ord(text[i])] = 1
            number_of_characters += 1
        elif present_characters[ord(text[i])] == 0:
            present_characters[ord(text[i])] = 1
            number_of_characters += 1
        if ord(text[i]) > max_character_order_index:
            max_character_order_index = ord(text[i])
        if ord(text[i]) < min_character_order_index:
            min_character_order_index = ord(text[i])
    return not_one_byte_counter, min_character_order_index, max_character_order_index, number_of_characters, present_characters

if not os.path.exists('enwik8_filtered'):
    if not os.path.exists('enwik8'):
        filename = maybe_download('enwik8.zip', 36445475)
    full_text = read_data(filename)
    new_text = u""
    new_text_list = list()
    for i in range(len(full_text)):
        if (i+1) % 10000000 == 0:
            print("%s characters are filtered" % i)
        if ord(full_text[i]) < 256:
            new_text_list.append(full_text[i])
    text = new_text.join(new_text_list)
    del new_text_list
    del new_text
    del full_text

    (not_one_byte_counter, min_character_order_index, max_character_order_index, number_of_characters, present_characters_indices) = check_not_one_byte(text)

    print("number of not one byte characters: ", not_one_byte_counter) 
    print("min order index: ", min_character_order_index)
    print("max order index: ", max_character_order_index)
    print("total number of characters: ", number_of_characters)
    
    f = open('enwik8_filtered', 'w')
    f.write(text.encode('utf8'))
    f.close()
    
else:
    f = open('enwik8_filtered', 'r')
    text = f.read().decode('utf8')
    f.close() 
    (not_one_byte_counter, min_character_order_index, max_character_order_index, number_of_characters, present_characters_indices) = check_not_one_byte(text)

    print("number of not one byte characters: ", not_one_byte_counter) 
    print("min order index: ", min_character_order_index)
    print("max order index: ", max_character_order_index)
    print("total number of characters: ", number_of_characters)    

In [ ]:
valid_size = 10000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

In [ ]:
#different
offset = 0
valid_size = 10000
valid_text = text[offset:offset+valid_size]
train_text = text[offset+valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

In [ ]:
vocabulary_size = number_of_characters
vocabulary = list()
characters_positions_in_vocabulary = list()

character_position_in_vocabulary = 0
for i in range(256):
    if present_characters_indices[i]:
        vocabulary.append(unichr(i))
        characters_positions_in_vocabulary.append(character_position_in_vocabulary)
        character_position_in_vocabulary += 1
    else:
        characters_positions_in_vocabulary.append(-1)

def char2id(char):
  if characters_positions_in_vocabulary[ord(char)] != -1:
    return characters_positions_in_vocabulary[ord(char)]
  else:
    print(u'Unexpected character: %s\nUnexpected character number: %s\nUnexpected character has its place = %s\n' % (char, ord(char), present_characters_indices[i]))
    return characters_positions_in_vocabulary[ord(char)]
  
def id2char(dictid):
  if (dictid >= 0) and (dictid < vocabulary_size):
    return vocabulary[dictid]
  else:
    print(u"unexpected id")
    return u'\0'

string_vocabulary = u""
for i in range(vocabulary_size):
    string_vocabulary += vocabulary[i]
print("Vocabulary: ", string_vocabulary)
print("char2id(u'a') = %s,  char2id(u'z') = %s,  char2id(u' ') = %s" % (char2id(u'a'), char2id(u'z'), char2id(u' ')))
print("id2char(78) = %s,  id2char(156) = %s,  id2char(140) = %s" % (id2char(78), id2char(156), id2char(140)))


In [ ]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [u""] * batches[0].shape[0]
  for b in batches:
    s = [u"".join(x) for x in zip(s, characters(b))]
  return s

train_batches_test = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches_test = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches_test.next()))
print(batches2string(train_batches_test.next()))
print(batches2string(valid_batches_test.next()))
print(batches2string(valid_batches_test.next()))

In [ ]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

def percent_of_correct_predictions(predictions, labels):
    num_characters = predictions.shape[0]
    num_correct = 0
    for i in range(num_characters):
        if labels[i, np.argmax(predictions, axis=1)[i]] == 1:
            num_correct += 1
    return float(num_correct) / num_characters * 100

In [ ]:
num_layers = 1
num_nodes = [64]

half_life = 3000
decay = 0.6

graph = tf.Graph()
with graph.as_default(): 
    with graph.device('/gpu:0'): 
        #list of LSTM cells in multilayered network
        cell_list = list()
        for i in range(num_layers):
            cell_list.append(tf.nn.rnn_cell.BasicLSTMCell(num_nodes[i], state_is_tuple=True))
        LSTM = tf.nn.rnn_cell.MultiRNNCell(cell_list, state_is_tuple=True)

        weights_initializer = tf.truncated_normal_initializer(mean=-0.1, stddev=0.1, seed=None, dtype=tf.float32)
        LSTM_scope = 'LSTM'
        with tf.variable_scope(LSTM_scope, reuse=False):
          for i in range(num_layers):
            with tf.variable_scope("Cell%d" % i):
              with tf.variable_scope("BasicLSTMCell"):
                with tf.variable_scope("Linear"):
                  if i == 0:
                    input_dim = vocabulary_size + cell_list[i].output_size
                  else:
                    input_dim = cell_list[i-1].output_size + cell_list[i].output_size
                  output_dim = 4*cell_list[i].output_size
                  tf.get_variable("Matrix", [input_dim, output_dim], initializer=weights_initializer)
                  tf.get_variable("Bias", 
                                  initializer=tf.zeros_initializer(shape=[output_dim], dtype=tf.float32))

        # classifier 
        weights = tf.Variable(tf.truncated_normal([LSTM.output_size, vocabulary_size], stddev = 0.1))
        bias = tf.Variable(tf.zeros([vocabulary_size]))

        train_data = list()
        for _ in range(num_unrollings + 1):
            train_data.append(
                tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
        train_inputs = train_data[:num_unrollings]
        train_labels = train_data[1:]  # labels are inputs shifted by one time step.
        # Unrolled LSTM loop.

        state = list()
        saved_state = list()
        for i in range(num_layers):
            zero_c, zero_h = cell_list[i].zero_state(batch_size, tf.float32)
            shape_c = zero_c.get_shape().as_list()
            shape_h = zero_h.get_shape().as_list()
            state.append((tf.zeros(shape_c), tf.zeros(shape_h))) 
            saved_state.append((tf.Variable(tf.zeros(shape_c)), tf.Variable(tf.zeros(shape_h))))

        outputs = list()
        state = saved_state
        for i in train_inputs:
            with tf.variable_scope('', reuse=True): 
                output, state = LSTM(i, state, scope=LSTM_scope)
            outputs.append(output)

        save_list = list()
        for i in range(num_layers):
            save_list.append(saved_state[i][0].assign(state[i][0]))
            save_list.append(saved_state[i][1].assign(state[i][1]))      

        with tf.control_dependencies(save_list):
                # Classifier.
            logits = tf.nn.xw_plus_b(tf.concat(0, outputs), weights, bias)
            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(
                logits, tf.concat(0, train_labels)))
        # Optimizer.
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(
            10.0, global_step, half_life, decay, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(
            zip(gradients, v), global_step=global_step)

        train_prediction = tf.nn.softmax(logits)

        # Sampling and validation eval: batch 1, no unrolling.
        saved_sample_state = list()

        for i in range(num_layers):
            sample_zero_c, sample_zero_h = cell_list[i].zero_state(1, tf.float32)
            sample_shape_c = sample_zero_c.get_shape().as_list()
            sample_shape_h = sample_zero_h.get_shape().as_list()
            saved_sample_state.append((tf.Variable(tf.zeros(sample_shape_c)), tf.Variable(tf.zeros(sample_shape_h))))  

        sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])

        reset_list = list()
        for i in range(num_layers):
            reset_list.append(saved_sample_state[i][0].assign(tf.zeros(sample_shape_c)))
            reset_list.append(saved_sample_state[i][1].assign(tf.zeros(sample_shape_h)))

        reset_sample_state = tf.group(*reset_list)

        with tf.variable_scope('', reuse=True): 
            sample_output, sample_state = LSTM(sample_input, saved_sample_state, scope=LSTM_scope)

        sample_save_list = list()
        for i in range(num_layers):
            sample_save_list.append(saved_sample_state[i][0].assign(sample_state[i][0]))
            sample_save_list.append(saved_sample_state[i][1].assign(sample_state[i][1]))    

        with tf.control_dependencies(sample_save_list):
            sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, weights, bias))

In [ ]:
num_steps = 10001
num_train_points = 100
num_train_points_per_1_validation_point = 10
train_frequency = num_steps / num_train_points
averaging_number = 10
averaging_number = min(averaging_number, train_frequency)

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

data_for_plot = {'train': {'step': list(), 'percentage': list()},
                 'validation': {'step': list(), 'percentage': list()}}

config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
with tf.Session(graph=graph, config=config) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    start_time = time.clock()
    average_percentage_of_correct = 0.
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate],
                                            feed_dict=feed_dict)
        mean_loss += l
        if (step % train_frequency) == 0 or step > ((step / train_frequency + 1) * train_frequency - averaging_number):
            labels = np.concatenate(list(batches)[1:])
            average_percentage_of_correct += percent_of_correct_predictions(predictions, labels)
        if step % train_frequency == 0:
            if step > 0:
                average_percentage_of_correct /= averaging_number
            data_for_plot['train']['step'].append(step)
            data_for_plot['train']['percentage'].append(average_percentage_of_correct)
            if step % (train_frequency * num_train_points_per_1_validation_point) == 0:
                if step > 0:
                    mean_loss = mean_loss / (train_frequency * num_train_points_per_1_validation_point)
                # The mean loss is an estimate of the loss over the last few batches.
                print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
                mean_loss = 0
                print('Percentage_of correct: %.2f%%' % average_percentage_of_correct)
                if step % (train_frequency * num_train_points_per_1_validation_point * 10) == 0:
                    # Generate some samples.
                    print('=' * 80)
                    for _ in range(5):
                        feed = sample(random_distribution())
                        sentence = characters(feed)[0]
                        reset_sample_state.run()
                        for _ in range(79):
                            prediction = sample_prediction.eval({sample_input: feed})
                            feed = sample(prediction)
                            sentence += characters(feed)[0]
                        print(sentence)
                    print('=' * 80)
                # Measure validation set perplexity.
                reset_sample_state.run()
                valid_logprob = 0
                validation_percentage_of_correct = 0.
                for _ in range(valid_size):
                    b = valid_batches.next()
                    predictions = sample_prediction.eval({sample_input: b[0]})
                    validation_percentage_of_correct += percent_of_correct_predictions(predictions, b[1])
                    valid_logprob = valid_logprob + logprob(predictions, b[1])
                print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))
                validation_percentage_of_correct /= valid_size
                print('Validation percentage of correct: %.2f%%\n' % validation_percentage_of_correct)
                data_for_plot['validation']['step'].append(step)
                data_for_plot['validation']['percentage'].append(validation_percentage_of_correct)
            average_percentage_of_correct = 0.
    finish_time = time.clock()
        


In [ ]:
colors = {0: 'magenta',
          1: 'cyan',
          2: 'yellow',
          3: 'firebrick',
          4: 'blue',
          5: 'gray',
          6: 'm',
          7: 'green',
          8: 'darkgoldenrod',
          9: 'purple',
          10: 'k',
          11: 'r',
          12: '#E24A33',
          13: '#92C6FF', 
          14: '#0072B2',
          15: '#30a2da',
          16: '#4C72B0',
          17: '#8EBA42',
          18: '#6d904f'}

plt.figure(1)
plt.plot(data_for_plot['train']['step'], data_for_plot['train']['percentage'], colors[0])
plt.plot(data_for_plot['validation']['step'], data_for_plot['validation']['percentage'], colors[1])
plt.title('Computation took %.f seconds' % (finish_time - start_time))
plt.xlabel('step')
plt.ylabel('percentage of correct')
x1, x2, y1, y2 = plt.axis()
plt.text(x2 + 0.05 * (x2 - x1), (y2 - y1) * 0.75 + y1, 'train percentage',  va = 'center', ha = 'left', color=colors[0])
plt.text(x2 + 0.05 * (x2 - x1), (y2 - y1) * 0.25 + y1, 'validation percentage',  va = 'center', ha = 'left', color=colors[1])
nodes_string = "%s"
if num_layers > 1:
    for i in range(num_layers - 1):
        nodes_string += "_%s"
    nodes_string = '(' + nodes_string + ')'
plot_filename = "nl%s_;nn_%s;nu_%s;hl_%s;dc_%s;bs_%s;emb_%s" % (num_layers,
                                                                nodes_string,
                                                                num_unrollings,
                                                                half_life,
                                                                decay,
                                                                batch_size,
                                                                False)
print(plot_filename)
plot_filename = (plot_filename % tuple(num_nodes)) + '.png'
plt.savefig(plot_filename)
plt.show()
plt.close()
